In [1]:
import pandas as pd
acctDF = pd.read_parquet("data/q2-ucsd-acctDF.pqt")
cat_map = pd.read_csv('data/q2-ucsd-cat-map.csv')
consDF = pd.read_parquet('data/q2-ucsd-consDF.pqt')
trxnDF = pd.read_parquet('data/q2-ucsd-trxnDF.pqt')

In [3]:
def credit_percentage():
    df = trxnDF.copy()
    credit = df[df.credit_or_debit == 'CREDIT'].drop(columns=['credit_or_debit']).groupby('prism_consumer_id').amount.sum().reset_index().rename(columns={'amount' : 'credit_amount'})
    debit = df[df.credit_or_debit == 'DEBIT'].drop(columns=['credit_or_debit']).groupby('prism_consumer_id').amount.sum().reset_index().rename(columns={'amount' : 'debit_amount'})
    normal_df = credit.merge(debit, on='prism_consumer_id', how='outer')
    normal_df['credit_percentage'] = normal_df['credit_amount'] / (normal_df['credit_amount'] + normal_df['debit_amount'])
    normal_df = normal_df.dropna()
    return normal_df
credit_percentage()

,prism_consumer_id,credit_amount,debit_amount,credit_percentage
0,0,14386.82,14908.41,0.491098
1,1,24903.80,23098.37,0.518806
2,10,20576.56,21766.60,0.485948
3,100,35236.84,39742.61,0.469953
4,1000,78353.07,77914.99,0.501402
...,...,...,...,...
14487,9995,16925.84,17047.39,0.498211
14488,9996,1200.03,1190.89,0.501911
14489,9997,17206.11,16702.92,0.507420
14490,9998,14566.37,15399.47,0.486099


In [17]:
def all_cat_percent():
    def get_cat_percent(df, category='all'):
        user_cat = df.groupby(['prism_consumer_id', 'category']).amount.sum().reset_index()
        pivot_df = user_cat.pivot(index='prism_consumer_id', columns='category', values='amount')
        pivot_df.fillna(0, inplace=True)
        pivot_df['total'] = pivot_df.sum(axis=1) 
        for i in (user_cat.category.unique()):
            pivot_df[i] = (pivot_df[i]/pivot_df['total']) * 100
            pivot_df = pivot_df.rename(columns={i : cat_map.loc[i].category})
        cat_percent = consDF.merge(pivot_df, on='prism_consumer_id', how='outer').drop(columns=['evaluation_date', 'credit_score'])
        cat_percent = cat_percent.drop(columns =['DQ_TARGET'])
        cat_percent = cat_percent.set_index('prism_consumer_id')
        column_names = []
        for i in cat_percent.columns:
            column_names.append(f'{category}_{i}')
        cat_percent.columns = column_names
        return cat_percent
    trxn = trxnDF.copy()
    credit_trxn = trxn[trxn['credit_or_debit'] == 'CREDIT']
    debit_trxn = trxn[trxn['credit_or_debit'] == 'DEBIT']
    
    credit_cat_features = get_cat_percent(credit_trxn, 'credit')
    debit_cat_features = get_cat_percent(debit_trxn, 'debit')
    trxn_cat_features = get_cat_percent(trxn, 'trxn')

    all_cat_features = credit_cat_features.merge(
        debit_cat_features, left_index = True, right_index = True)
    all_cat_features = all_cat_features.merge(
        trxn_cat_features, left_index = True, right_index = True)
    return all_cat_features
all_cat_percent()

,credit_SELF_TRANSFER,credit_EXTERNAL_TRANSFER,credit_DEPOSIT,credit_PAYCHECK,credit_MISCELLANEOUS,credit_REFUND,credit_INVESTMENT_INCOME,credit_OTHER_BENEFITS,credit_UNEMPLOYMENT_BENEFITS,credit_TAX,...,trxn_LEGAL,trxn_GOVERNMENT_SERVICES,trxn_RISK_CATCH_ALL,trxn_RTO_LTO,trxn_INVESTMENT,trxn_GAMBLING,trxn_CORPORATE_PAYMENTS,trxn_TIME_OR_STUFF,trxn_PENSION,trxn_total
prism_consumer_id,,,,,,,,,,,,,,,,,,,,,
0,15.377964,1.589997,3.475403,61.310005,0.011330,0.138738,0.0000,0.0,0.0,18.096563,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,29295.23
1,36.552655,0.000000,5.994868,47.858720,0.246509,0.009717,0.0000,0.0,0.0,9.337531,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,48002.17
10,9.236481,15.337841,3.401929,71.541307,0.033728,0.448714,0.0000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.392035,0.0,0.0,0.0,42343.16
100,21.994027,8.684093,0.000000,69.279141,0.009394,0.033346,0.0000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,74979.45
1000,22.663617,1.712977,6.024014,55.720344,0.330453,0.001748,0.0000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,7.496330,0.000000,0.0,0.0,0.0,156268.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,9.015801,0.000000,66.329588,0.000000,0.000000,0.0000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,33973.23
9996,0.000000,0.000000,0.000000,0.000000,99.997500,0.000000,0.0025,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,2390.92
9997,0.000000,1.152207,0.235730,96.665080,0.000000,1.946983,0.0000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,33909.03


In [23]:
def running_total():
    def running_total_statistics(df, time_frame='1D'):
        df['posted_date'] = pd.to_datetime(df['posted_date'])
        df.loc[df['credit_or_debit'] == 'DEBIT', 'amount'] *= -1 
        df = df.groupby(['prism_consumer_id', pd.Grouper(key='posted_date', freq=time_frame)]).amount.sum().reset_index()
        df = df.sort_values('posted_date') 
        df['running_total'] = df.groupby('prism_consumer_id')['amount'].cumsum()
        new_df = df.groupby('prism_consumer_id').agg({'running_total': ['mean', 'median', 'var']})
        new_df.columns = [f'{time_frame}_mean', f'{time_frame}_median', f'{time_frame}_var']
        return new_df
    month = running_total_statistics(trxnDF.copy(), time_frame='1M')
    week = running_total_statistics(trxnDF.copy(), time_frame='1W')
    day = running_total_statistics(trxnDF.copy(), time_frame='1D')
    all_features = month.merge(week, left_index=True, right_index=True)
    all_features = all_features.merge(day, left_index=True, right_index=True)
    return all_features
running_total()

/tmp/ipykernel_383/2494711840.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.groupby(['prism_consumer_id', pd.Grouper(key='posted_date', freq=time_frame)]).amount.sum().reset_index()


,1M_mean,1M_median,1M_var,1W_mean,1W_median,1W_var,1D_mean,1D_median,1D_var
prism_consumer_id,,,,,,,,,
0,-126.028571,-521.590,1.304211e+06,55.769231,-135.165,1.180367e+06,-17.708462,-224.580,1.032843e+06
1,1577.304286,1805.430,1.029055e+06,1525.188519,1523.290,1.314699e+06,1583.293585,1667.110,1.344498e+06
10,-914.140000,-1110.080,2.179709e+05,-861.358846,-902.495,2.251006e+05,-929.185115,-921.150,2.517296e+05
100,-4442.633333,-4672.015,2.572535e+06,-4289.333846,-4597.015,4.100968e+06,-4108.325741,-4597.015,4.096265e+06
1000,740.154286,311.400,1.287469e+06,1029.141481,671.400,9.147447e+05,929.820633,520.170,2.406822e+06
...,...,...,...,...,...,...,...,...,...
9995,-78.102500,-82.385,1.678502e+03,228.592308,136.980,1.186324e+05,50.434286,-47.165,5.803736e+04
9996,-10.577500,-4.895,7.227368e+02,-70.760000,-38.670,5.614215e+03,-82.671724,-68.930,5.763918e+03
9997,496.465000,326.455,3.749928e+05,100.410000,-21.600,1.104554e+05,382.507302,251.970,3.123170e+05


In [25]:
def days_open():
    trxn_time = trxnDF.copy()
    trxn_time.loc[trxn_time['credit_or_debit'] == 'DEBIT', 'amount'] *= -1
    trxn_time = trxn_time.groupby(['prism_consumer_id']).agg(total_amount=('amount', 'sum'), 
                                                               first_date=('posted_date', 'min'), last_date=('posted_date','max'))
    trxn_time['last_date'] = pd.to_datetime(trxn_time['last_date'])
    trxn_time['first_date'] = pd.to_datetime(trxn_time['first_date'])
    trxn_time['total_days'] = (trxn_time['last_date'] - trxn_time['first_date']).dt.days
    return trxn_time[['total_days']]
days_open()

,total_days
prism_consumer_id,
0,179
1,180
10,180
100,179
1000,180
...,...
9995,86
9996,78
9997,87
